In [17]:
!pip install tweepy
!pip3 install -U scikit-learn


In [ ]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
from pymongo import MongoClient
import json

In [6]:
consumer_key=''

In [7]:
consumer_secret=''

In [8]:
access_token=''

In [9]:
access_token_secret=''

In [10]:
auth= OAuthHandler(consumer_key, consumer_secret)

In [12]:
auth.set_access_token(access_token, access_token_secret)

In [14]:
# criando uma classe para capturar os stream de dados do tweeter e
# armazenar no MongoDB

class MyListener(StreamListener):
    def on_data(self, dados):
        tweet= json.loads(dados)
        created_at= tweet['created_at']
        id_str= tweet['id_str']
        text= tweet['text']
        obj= {'created_at':created_at,'id_str':id_str,'text':text,}
        tweetind= col.insert_one(obj).inserted_id
        print(obj)
        return True

In [15]:
myListener= MyListener()

In [16]:
mystream= Stream(auth, listener= myListener)

## conectando com o Mongo

In [18]:
client_con= MongoClient('localhost', 6660)

In [19]:
db= client_con.twitterdb

In [20]:
col= db.tweets

In [21]:
keywords= ['Big Data', 'Python', 'Data Mining', 'Data Science']

In [ ]:
mystream.filter(track=keywords)

In [24]:
mystream.disconnect()

## Usando o pandas e scikit-learn para analisar os dados coletados

In [41]:
#criando o dataset com os dados retornados do mongoDB
dataset= [{'created_at':item ['created_at'], 'text':item['text'],} for item in col.find()]

In [42]:
import pandas as pd

In [43]:
df= pd.DataFrame(dataset)

In [ ]:
df

In [48]:
from sklearn.feature_extraction.text import CountVectorizer

In [51]:
cv= CountVectorizer()

In [52]:
count_matrix= cv.fit_transform(df.text)

In [ ]:
word_count= pd.DataFrame(cv.get_feature_names(), columns=['word'])
word_count['count']= count_matrix.sum(axis=0).tolist()[0]
word_count= word_count.sort_values('count', ascending=False).reset_index(drop=True)
word_count[:50]

# Fim